In [1]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Look ONLY for these five things:

1. Root: the beginning part of all URLs associated with this API for getting data back (I often find this in examples first)

https://api.congress.gov/v3

2. Endpoint: the second part of the URL, generally it points to a specific data set to be returned

/member/{bioguideId}

The curly braces mean: fill this in for whichever member of congress you want

3. The headers: data that WE give to the API to identify ourselves, and provide other information if requested
 * User Agent string: a string that identifies your software. If you are writing a Bot, what is your bot's name, and what methods are you using to access this API.

 The standard convention is:

 'botname/version (email) python-requests/request-package-version'



In [2]:
botname = 'targ'
version = '0.0'
email = 'jkropko@virginia.edu'
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent':useragent}
headers

{'User-Agent': 'targ/0.0 (jkropko@virginia.edu) python-requests/2.32.5'}

4. API Parameters: these are ways to control what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameter:

* Query parameters: should be listed in a dictionary and passed to the `params` argument of `requests.get()`

* Path parameters: are part of the endpoint (such as bioguideId)

* Global parameters: apply to any use of the API regardless of endpoint. Often that includes your API key. Can be in either the headers or the params

5. If an API Key is needed, how to get one, and how will you let the API know the key?

API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your key.

In [3]:
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')

In [4]:
params = {'format': 'json',
          'api_key': congresskey} 

Put it all together.

In [5]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint, 
                headers = headers,
                params = params)
r

<Response [200]>

In [6]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 168,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

## Sponsored legislation

In [7]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
endpoint

'/member/M001239/sponsored-legislation'

In [9]:
params = {'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}


In [10]:
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

In [12]:
myjson = r.json()

In [18]:
#myjson['sponsoredLegislation']

sponsoredLegislation_df = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])

In [23]:
for x in sponsoredLegislation_df['title']:
    print(x)

To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.
To change the name of the Department of Defense to the "Department of War", and for other purposes.
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [25]:
r = requests.get(sponsoredLegislation_df['url'][3],
                 headers=headers,
                 params=params)
r

<Response [200]>

In [26]:
r.json()

{'bill': {'actions': {'count': 6,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/actions?format=json'},
  'committees': {'count': 2,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 144 (Wednesday, September 3, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5103.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H3829]\n</pre>',
  'cosponsors': {'count': 3,
   'countIncludingWithdrawnCosponsors': 3,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/cosponsors?format=json'},
  'introducedDate': '2025-09-03',
  'latestAction': {'actionDate': '2025-09-10',
   'text': 'Ordered to be Reported (Amended) by the Yeas and Nays: 25 - 19.'},
  'legislationUrl': 'https:

## FEC Data for financial campaign contributions

In [ ]:
dotenv.load_dotenv()
feckey = os.getenv('feckey')

In [45]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/candidates/search/'

params = {'api_key': feckey,
          'q': 'John McGuire',
          'state': 'VA',
          'district': '5',
          'office': 'H'}
r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r

<Response [200]>

In [49]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [58]:
endpoint = f'/v1/candidate/{candidate_id}/committees/'
params = {'api_key': feckey}

r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r   

<Response [200]>

In [62]:
committee_ids = [x['committee_id'] for x in r.json()['results']]
committee_ids

['C00727594', 'C00856831', 'C00900118']

In [65]:
endpoint = '/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'committee_id': committee_ids[1],
          'two_year_transaction_period': '2026'}

r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r

<Response [200]>

In [76]:
contrib_dict = [{'contributor_name': x['contributor_name'],
                'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                'memo_text': x['memo_text']} for x in r.json()['results']]
pd.DataFrame(contrib_dict)

,contributor_name,contributor_aggregate_ytd,memo_text
0,MCGUIRE VICTORY FUND,72433.73,TRANSFER FROM JOINT FUNDRAISER
1,REPUBLICAN MAINSTREET PARTNERSHIP PAC,1000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
2,"STOUDT, PETER A",2000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
3,"STINSON, BEVERLEY",1000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
4,"GRANT, PETER M II",5000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
5,"COLE, JON C",1000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
6,"CARY, ROBERT M",2500.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...
7,MCGUIRE VICTORY FUND,72433.73,TRANSFER FROM JOINT FUNDRAISER
8,MCGUIRE VICTORY FUND,72433.73,TOTAL OF UNITEMIZED CONTRIBUTIONS PART OF A TR...
9,"GRANT, PETER M II",5000.00,MEMO TRANSFER FROM JOINT FUNDRAISER-MCGUIRE VI...


In [72]:
myjson

{'amendment_indicator': 'N',
 'amendment_indicator_desc': 'NO CHANGE',
 'back_reference_schedule_name': None,
 'back_reference_transaction_id': 'A-3184',
 'candidate_first_name': None,
 'candidate_id': None,
 'candidate_last_name': None,
 'candidate_middle_name': None,
 'candidate_name': None,
 'candidate_office': None,
 'candidate_office_district': None,
 'candidate_office_full': None,
 'candidate_office_state': None,
 'candidate_office_state_full': None,
 'candidate_prefix': None,
 'candidate_suffix': None,
 'committee': {'affiliated_committee_name': 'MCGUIRE VICTORY FUND',
  'candidate_ids': ['H0VA07133'],
  'city': 'MANAKIN SABOT',
  'committee_id': 'C00856831',
  'committee_type': 'H',
  'committee_type_full': 'House',
  'cycle': 2026,
  'cycles': [2024, 2026],
  'cycles_has_activity': [2024, 2026],
  'cycles_has_financial': [2024, 2026],
  'designation': 'P',
  'designation_full': 'Principal campaign committee',
  'filing_frequency': 'Q',
  'first_f1_date': '2023-11-15',
  'first